In [ ]:
import pandas as pd
import numpy as np
import tempfile
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def FrontPage():
  # Loading the orders data frame
  ords = pd.read_csv('drive/My Drive/Flipkart Grid/orders.csv').drop(columns = ['Unnamed: 0'])
  ords['User'] = ords['User'].astype(str)
  ords['Item ID'] = ords['Item ID'].astype(str)

  # Loading the products data frame
  products = pd.read_csv('drive/My Drive/Flipkart Grid/products.csv').drop(columns = ['Unnamed: 0']) 
  products['Item ID'] = products['Item ID'].astype(str)

  display = {}
  
  # Most Purchased Products
  display['Popular'] = ords[['Order ID', 'Item ID']].groupby(by = 'Item ID').count().sort_values(by = 'Order ID', ascending = False).index.tolist()[:10]
  
  # Biggest Discounts
  discounts = products.nlargest(10, ['Discount']).values
  display['Top Discounts'] = [i[0] for i in discounts]

  # Sponsored products
  sponsored = products[products['Sponsored'] == True].sample(frac = 1).values[:10]
  display['Sponsored'] = [i[0] for i in sponsored]

  return display
FrontPage()

{'Popular': ['153', '127', '61', '87', '81', '101', '77', '102', '195', '78'],
 'Sponsored': ['202',
  '26',
  '279',
  '105',
  '267',
  '124',
  '269',
  '85',
  '141',
  '178'],
 'Top Discounts': ['131',
  '166',
  '183',
  '277',
  '165',
  '2',
  '12',
  '28',
  '31',
  '36']}

In [ ]:
def SmartBagRecommendations(user_number):
  with tempfile.TemporaryDirectory() as tmp:
    path = 'drive/My Drive/Flipkart Grid/TFRS_model'

    # Load it back; can also be done in TensorFlow Serving.
    loaded = tf.saved_model.load(path)

    # Pass a user id in, get top predicted products back.
    scores, titles = loaded([user_number])
    
    # Loading the orders data frame
    ords = pd.read_csv('drive/My Drive/Flipkart Grid/orders.csv').drop(columns = ['Unnamed: 0'])
    ords['User'] = ords['User'].astype(str)
    ords['Item ID'] = ords['Item ID'].astype(str)

    # Loading the products data frame
    products = pd.read_csv('drive/My Drive/Flipkart Grid/products.csv').drop(columns = ['Unnamed: 0']) 
    products['Item ID'] = products['Item ID'].astype(str)

    rec_ids = []
    for rec in titles[0]:
      rec_ids.append(products[products['Item ID'] == np.array(rec).astype(str)].values[0][0])

    recommendations = {}
    if len(ords[ords['User'] == user_number]) > 0:
      # Initializing the dictionary of labeled predictions

      # Adding model's top 10 predictions to the 'top' list
      recommendations['top'] = rec_ids

      # Adding the 2 most frequently ordered products
      user_data = ords[ords.User == user_number]
      frequent = user_data['Item ID'].value_counts().index.tolist()[:3]
      values = list(user_data['Item ID'].value_counts())
      recommendations['most_bought'] = frequent

      # Adding discount/larger/sponsored product recommendations
      recommendations['discounted'], recommendations['bigger'], recommendations['sponsored'] = [], [], []
      user_data = user_data.merge(products, on = 'Item ID')
      freq_types = user_data['Type'].value_counts().index.tolist()[:3]
      for ptype in freq_types:
        subset = products[products['Type'] == ptype]
        disc = str(subset['Discount'].idxmax())
        biggest = str(subset['Quantity'].idxmax())
        spons = subset[subset['Sponsored'] == True].values
        if subset[subset['Item ID'] == disc].values[0][-3]: recommendations['discounted'].append(disc)
        recommendations['bigger'].append(biggest)
        if len(spons) > 0: recommendations['sponsored'].append(spons[0][0])

    else:
      recommendations = {'model_based': rec_ids}
      pop = ords[['Order ID', 'Item ID']].groupby(by = 'Item ID').count().sort_values(by = 'Order ID', ascending = False).index.tolist()[:10]
      recommendations['data_based'] = pop
    return recommendations

In [ ]:
%%time
SmartBagRecommendations("777")

CPU times: user 313 ms, sys: 16.4 ms, total: 330 ms
Wall time: 338 ms


{'bigger': ['246', '38', '54'],
 'discounted': ['246', '36', '54'],
 'most_bought': ['40', '52', '119'],
 'sponsored': ['249'],
 'top': ['288', '163', '40', '100', '192', '29', '250', '252', '178', '52']}

In [ ]:
%%time
SmartBagRecommendations("2000")

CPU times: user 281 ms, sys: 8.91 ms, total: 290 ms
Wall time: 299 ms


{'data_based': ['153',
  '127',
  '61',
  '87',
  '81',
  '101',
  '77',
  '102',
  '195',
  '78'],
 'model_based': ['74',
  '176',
  '189',
  '279',
  '22',
  '186',
  '89',
  '281',
  '37',
  '26']}